In [147]:
import pandas as pd 
from pymongo import MongoClient
import numpy as np
import ast
from collections import defaultdict
import json
from operator import itemgetter
import nltk
import sys

In [2]:
word_data1=pd.read_csv("vedat_result.csv")
word_data1.head()

,Unnamed: 0,Title,Intro,Plot,Title_tokenized,Intro_tokenized,Plot_tokenized,Title_lemmatize,Intro_lemmatize,Plot_lemmatize,Title_sw,Intro_sw,Plot_sw,Title_punc,Intro_punc,Plot_punc
0,0,love by the light of the moon - wikipedia,love by the light of the moon is a 1901 film b...,"the moon, painted with a smiling face hangs ov...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['the', 'moon', 'painted', 'with', 'a', 'smili...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['the', 'moon', 'painted', 'with', 'a', 'smili...","['love', 'light', 'moon', 'wikipedia']","['love', 'light', 'moon', '1901', 'film', 'edw...","['moon', 'painted', 'smiling', 'face', 'hang',...","['love', 'light', 'moon', 'wikipedia']","['love', 'light', 'moon', '1901', 'film', 'edw...","['moon', 'painted', 'smiling', 'face', 'hang',..."
1,1,the martyred presidents - wikipedia,the martyred presidents is a 1901 american fil...,"the film, just over a minute long, is composed...","['the', 'martyred', 'presidents', 'wikipedia']","['the', 'martyred', 'presidents', 'is', 'a', '...","['the', 'film', 'just', 'over', 'a', 'minute',...","['the', 'martyred', 'president', 'wikipedia']","['the', 'martyred', 'president', 'is', 'a', '1...","['the', 'film', 'just', 'over', 'a', 'minute',...","['martyred', 'president', 'wikipedia']","['martyred', 'president', '1901', 'american', ...","['film', 'minute', 'long', 'composed', 'two', ...","['martyred', 'president', 'wikipedia']","['martyred', 'president', '1901', 'american', ...","['film', 'minute', 'long', 'composed', 'two', ..."
2,2,"terrible teddy, the grizzly king - wikipedia","terrible teddy, the grizzly king is a 1901 ame...",lasting just 61 seconds and consisting of two ...,"['terrible', 'teddy', 'the', 'grizzly', 'king'...","['terrible', 'teddy', 'the', 'grizzly', 'king'...","['lasting', 'just', '61', 'seconds', 'and', 'c...","['terrible', 'teddy', 'the', 'grizzly', 'king'...","['terrible', 'teddy', 'the', 'grizzly', 'king'...","['lasting', 'just', '61', 'second', 'and', 'co...","['terrible', 'teddy', 'grizzly', 'king', 'wiki...","['terrible', 'teddy', 'grizzly', 'king', '1901...","['lasting', '61', 'second', 'consisting', 'two...","['terrible', 'teddy', 'grizzly', 'king', 'wiki...","['terrible', 'teddy', 'grizzly', 'king', '1901...","['lasting', '61', 'second', 'consisting', 'two..."
3,3,jack and the beanstalk (1902 film) - wikipedia,jack and the beanstalk is a 1902 american shor...,in this earliest known adaptation of the class...,"['jack', 'and', 'the', 'beanstalk', '1902', 'f...","['jack', 'and', 'the', 'beanstalk', 'is', 'a',...","['in', 'this', 'earliest', 'known', 'adaptatio...","['jack', 'and', 'the', 'beanstalk', '1902', 'f...","['jack', 'and', 'the', 'beanstalk', 'is', 'a',...","['in', 'this', 'earliest', 'known', 'adaptatio...","['jack', 'beanstalk', '1902', 'film', 'wikiped...","['jack', 'beanstalk', '1902', 'american', 'sho...","['earliest', 'known', 'adaptation', 'classic',...","['jack', 'beanstalk', '1902', 'film', 'wikiped...","['jack', 'beanstalk', '1902', 'american', 'sho...","['earliest', 'known', 'adaptation', 'classic',..."
4,4,alice in wonderland (1903 film) - wikipedia,"it is now available from several sources, and ...","alice follows a large white rabbit down a ""rab...","['alice', 'in', 'wonderland', '1903', 'film', ...","['it', 'is', 'now', 'available', 'from', 'seve...","['alice', 'follows', 'a', 'large', 'white', 'r...","['alice', 'in', 'wonderland', '1903', 'film', ...","['it', 'is', 'now', 'available', 'from', 'seve...","['alice', 'follows', 'a', 'large', 'white', 'r...","['alice', 'wonderland', '1903', 'film', 'wikip...","['available', 'several', 'source', 'included',...","['alice', 'follows', 'large', 'white', 'rabbit...","['alice', 'wonderland', '1903', 'film', 'wikip...","['available', 'several', 'source', 'included',...","['alice', 'follows', 'large', 'white', 'rabbit..."


In [3]:
word_data2=pd.read_csv("poo_result.csv")
word_data2.head()

,Unnamed: 0,Title,Intro,Plot,Title_tokenized,Intro_tokenized,Plot_tokenized,Title_lemmatize,Intro_lemmatize,Plot_lemmatize,Title_sw,Intro_sw,Plot_sw,Title_punc,Intro_punc,Plot_punc
0,0,10 to midnight - wikipedia,10 to midnight is a 1983 american crime-horror...,warren stacy (gene davis) is a young office eq...,"['10', 'to', 'midnight', 'wikipedia']","['10', 'to', 'midnight', 'is', 'a', '1983', 'a...","['warren', 'stacy', 'gene', 'davis', 'is', 'a'...","['10', 'to', 'midnight', 'wikipedia']","['10', 'to', 'midnight', 'is', 'a', '1983', 'a...","['warren', 'stacy', 'gene', 'davis', 'is', 'a'...","['10', 'midnight', 'wikipedia']","['10', 'midnight', '1983', 'american', 'film',...","['warren', 'stacy', 'gene', 'davis', 'young', ...","['10', 'midnight', 'wikipedia']","['10', 'midnight', '1983', 'american', 'film',...","['warren', 'stacy', 'gene', 'davis', 'young', ..."
1,1,all the right moves (film) - wikipedia,all the right moves is a 1983 american sports ...,"stefen ""stef"" djordjevic (cruise) is a serbian...","['all', 'the', 'right', 'moves', 'film', 'wiki...","['all', 'the', 'right', 'moves', 'is', 'a', '1...","['stefen', 'stef', 'djordjevic', 'cruise', 'is...","['all', 'the', 'right', 'move', 'film', 'wikip...","['all', 'the', 'right', 'move', 'is', 'a', '19...","['stefen', 'stef', 'djordjevic', 'cruise', 'is...","['right', 'move', 'film', 'wikipedia']","['right', 'move', '1983', 'american', 'sport',...","['stefen', 'stef', 'djordjevic', 'cruise', 'se...","['right', 'move', 'film', 'wikipedia']","['right', 'move', '1983', 'american', 'sport',...","['stefen', 'stef', 'djordjevic', 'cruise', 'se..."
2,2,americana (film) - wikipedia,"in 1981, the film won the people\'s choice awa...","a down on his luck, former green beret captain...","['americana', 'film', 'wikipedia']","['in', '1981', 'the', 'film', 'won', 'the', 'c...","['a', 'down', 'on', 'his', 'luck', 'former', '...","['americana', 'film', 'wikipedia']","['in', '1981', 'the', 'film', 'won', 'the', 'c...","['a', 'down', 'on', 'his', 'luck', 'former', '...","['americana', 'film', 'wikipedia']","['1981', 'film', 'choice', 'award', 'fortnight...","['luck', 'former', 'green', 'beret', 'captain'...","['americana', 'film', 'wikipedia']","['1981', 'film', 'choice', 'award', 'fortnight...","['luck', 'former', 'green', 'beret', 'captain'..."
3,3,amityville 3-d - wikipedia,due to a lawsuit between the lutz family and d...,after he exposes a pair of con artists with hi...,"['amityville', 'wikipedia']","['due', 'to', 'a', 'lawsuit', 'between', 'the'...","['after', 'he', 'exposes', 'a', 'pair', 'of', ...","['amityville', 'wikipedia']","['due', 'to', 'a', 'lawsuit', 'between', 'the'...","['after', 'he', 'expose', 'a', 'pair', 'of', '...","['amityville', 'wikipedia']","['due', 'lawsuit', 'lutz', 'family', 'dino', '...","['expose', 'pair', 'con', 'artist', 'partner',...","['amityville', 'wikipedia']","['due', 'lawsuit', 'lutz', 'family', 'dino', '...","['expose', 'pair', 'con', 'artist', 'partner',..."
4,4,anna to the infinite power - wikipedia,anna to the infinite power is a 1982 science-f...,"twelve-year-old anna hart of flemington, new j...","['anna', 'to', 'the', 'infinite', 'power', 'wi...","['anna', 'to', 'the', 'infinite', 'power', 'is...","['anna', 'hart', 'of', 'flemington', 'new', 'j...","['anna', 'to', 'the', 'infinite', 'power', 'wi...","['anna', 'to', 'the', 'infinite', 'power', 'is...","['anna', 'hart', 'of', 'flemington', 'new', 'j...","['anna', 'infinite', 'power', 'wikipedia']","['anna', 'infinite', 'power', '1982', 'thrille...","['anna', 'hart', 'flemington', 'new', 'jersey'...","['anna', 'infinite', 'power', 'wikipedia']","['anna', 'infinite', 'power', '1982', 'thrille...","['anna', 'hart', 'flemington', 'new', 'jersey'..."


In [4]:
word_data3=pd.read_csv("tommaso_result.csv")
word_data3.head()

,Unnamed: 0,Title,Intro,Plot,Title_tokenized,Intro_tokenized,Plot_tokenized,Title_lemmatize,Intro_lemmatize,Plot_lemmatize,Title_sw,Intro_sw,Plot_sw,Title_punc,Intro_punc,Plot_punc
0,0,z.p.g. - wikipedia,"z.p.g. (short for ""zero population growth"") is...","set in the future, the earth has become severe...",['wikipedia'],"['short', 'for', 'zero', 'population', 'growth...","['set', 'in', 'the', 'future', 'the', 'earth',...",['wikipedia'],"['short', 'for', 'zero', 'population', 'growth...","['set', 'in', 'the', 'future', 'the', 'earth',...",['wikipedia'],"['short', 'zero', 'population', 'growth', '197...","['set', 'future', 'earth', 'ha', 'become', 'se...",['wikipedia'],"['short', 'zero', 'population', 'growth', '197...","['set', 'future', 'earth', 'ha', 'become', 'se..."
1,1,zee and co. - wikipedia,"the theme song ""going in circles"" was covered ...","zee blakely (elizabeth taylor) is a loud, coar...","['zee', 'and', 'wikipedia']","['the', 'theme', 'song', 'going', 'in', 'circl...","['zee', 'blakely', 'elizabeth', 'taylor', 'is'...","['zee', 'and', 'wikipedia']","['the', 'theme', 'song', 'going', 'in', 'circl...","['zee', 'blakely', 'elizabeth', 'taylor', 'is'...","['zee', 'wikipedia']","['theme', 'song', 'going', 'circle', 'wa', 'co...","['zee', 'blakely', 'elizabeth', 'taylor', 'lou...","['zee', 'wikipedia']","['theme', 'song', 'going', 'circle', 'wa', 'co...","['zee', 'blakely', 'elizabeth', 'taylor', 'lou..."
2,2,and now the screaming starts! - wikipedia,NaN,NaN,"['and', 'now', 'the', 'screaming', 'starts', '...",[],[],"['and', 'now', 'the', 'screaming', 'start', 'w...",[],[],"['screaming', 'start', 'wikipedia']",[],[],"['screaming', 'start', 'wikipedia']",[],[]
3,3,the asphyx - wikipedia,the asphyx is a 1972 british horror film[1] di...,"in victorian england, philanthropic scientist ...","['the', 'asphyx', 'wikipedia']","['the', 'asphyx', 'is', 'a', '1972', 'british'...","['in', 'victorian', 'england', 'philanthropic'...","['the', 'asphyx', 'wikipedia']","['the', 'asphyx', 'is', 'a', '1972', 'british'...","['in', 'victorian', 'england', 'philanthropic'...","['asphyx', 'wikipedia']","['asphyx', '1972', 'british', 'horror', 'film'...","['victorian', 'england', 'philanthropic', 'sci...","['asphyx', 'wikipedia']","['asphyx', '1972', 'british', 'horror', 'film'...","['victorian', 'england', 'philanthropic', 'sci..."
4,4,assassin (1973 film) - wikipedia,assassin is a 1973 british thriller film direc...,the british government hires an assassin to ki...,"['assassin', '1973', 'film', 'wikipedia']","['assassin', 'is', 'a', '1973', 'british', 'th...","['the', 'british', 'government', 'hires', 'an'...","['assassin', '1973', 'film', 'wikipedia']","['assassin', 'is', 'a', '1973', 'british', 'th...","['the', 'british', 'government', 'hire', 'an',...","['assassin', '1973', 'film', 'wikipedia']","['assassin', '1973', 'british', 'thriller', 'f...","['british', 'government', 'hire', 'assassin', ...","['assassin', '1973', 'film', 'wikipedia']","['assassin', '1973', 'british', 'thriller', 'f...","['british', 'government', 'hire', 'assassin', ..."


In [5]:
data=pd.concat([word_data1,word_data2,word_data3],ignore_index=True)
data.drop(["Unnamed: 0"],axis=1 ,inplace=True)
data.index += 1

In [6]:
data.head()

,Title,Intro,Plot,Title_tokenized,Intro_tokenized,Plot_tokenized,Title_lemmatize,Intro_lemmatize,Plot_lemmatize,Title_sw,Intro_sw,Plot_sw,Title_punc,Intro_punc,Plot_punc
1,love by the light of the moon - wikipedia,love by the light of the moon is a 1901 film b...,"the moon, painted with a smiling face hangs ov...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['the', 'moon', 'painted', 'with', 'a', 'smili...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['the', 'moon', 'painted', 'with', 'a', 'smili...","['love', 'light', 'moon', 'wikipedia']","['love', 'light', 'moon', '1901', 'film', 'edw...","['moon', 'painted', 'smiling', 'face', 'hang',...","['love', 'light', 'moon', 'wikipedia']","['love', 'light', 'moon', '1901', 'film', 'edw...","['moon', 'painted', 'smiling', 'face', 'hang',..."
2,the martyred presidents - wikipedia,the martyred presidents is a 1901 american fil...,"the film, just over a minute long, is composed...","['the', 'martyred', 'presidents', 'wikipedia']","['the', 'martyred', 'presidents', 'is', 'a', '...","['the', 'film', 'just', 'over', 'a', 'minute',...","['the', 'martyred', 'president', 'wikipedia']","['the', 'martyred', 'president', 'is', 'a', '1...","['the', 'film', 'just', 'over', 'a', 'minute',...","['martyred', 'president', 'wikipedia']","['martyred', 'president', '1901', 'american', ...","['film', 'minute', 'long', 'composed', 'two', ...","['martyred', 'president', 'wikipedia']","['martyred', 'president', '1901', 'american', ...","['film', 'minute', 'long', 'composed', 'two', ..."
3,"terrible teddy, the grizzly king - wikipedia","terrible teddy, the grizzly king is a 1901 ame...",lasting just 61 seconds and consisting of two ...,"['terrible', 'teddy', 'the', 'grizzly', 'king'...","['terrible', 'teddy', 'the', 'grizzly', 'king'...","['lasting', 'just', '61', 'seconds', 'and', 'c...","['terrible', 'teddy', 'the', 'grizzly', 'king'...","['terrible', 'teddy', 'the', 'grizzly', 'king'...","['lasting', 'just', '61', 'second', 'and', 'co...","['terrible', 'teddy', 'grizzly', 'king', 'wiki...","['terrible', 'teddy', 'grizzly', 'king', '1901...","['lasting', '61', 'second', 'consisting', 'two...","['terrible', 'teddy', 'grizzly', 'king', 'wiki...","['terrible', 'teddy', 'grizzly', 'king', '1901...","['lasting', '61', 'second', 'consisting', 'two..."
4,jack and the beanstalk (1902 film) - wikipedia,jack and the beanstalk is a 1902 american shor...,in this earliest known adaptation of the class...,"['jack', 'and', 'the', 'beanstalk', '1902', 'f...","['jack', 'and', 'the', 'beanstalk', 'is', 'a',...","['in', 'this', 'earliest', 'known', 'adaptatio...","['jack', 'and', 'the', 'beanstalk', '1902', 'f...","['jack', 'and', 'the', 'beanstalk', 'is', 'a',...","['in', 'this', 'earliest', 'known', 'adaptatio...","['jack', 'beanstalk', '1902', 'film', 'wikiped...","['jack', 'beanstalk', '1902', 'american', 'sho...","['earliest', 'known', 'adaptation', 'classic',...","['jack', 'beanstalk', '1902', 'film', 'wikiped...","['jack', 'beanstalk', '1902', 'american', 'sho...","['earliest', 'known', 'adaptation', 'classic',..."
5,alice in wonderland (1903 film) - wikipedia,"it is now available from several sources, and ...","alice follows a large white rabbit down a ""rab...","['alice', 'in', 'wonderland', '1903', 'film', ...","['it', 'is', 'now', 'available', 'from', 'seve...","['alice', 'follows', 'a', 'large', 'white', 'r...","['alice', 'in', 'wonderland', '1903', 'film', ...","['it', 'is', 'now', 'available', 'from', 'seve...","['alice', 'follows', 'a', 'large', 'white', 'r...","['alice', 'wonderland', '1903', 'film', 'wikip...","['available', 'several', 'source', 'included',...","['alice', 'follows', 'large', 'white', 'rabbit...","['alice', 'wonderland', '1903', 'film', 'wikip...","['available', 'several', 'source', 'included',...","['alice', 'follows', 'large', 'white', 'rabbit..."


In [7]:
client = MongoClient("127.0.0.1",27017)
db=client["HW3"]
indexes=[]
info_box=db["LastData"].find()
info_box=pd.DataFrame(info_box)
info_box.drop(["_id"],axis=1,inplace=True)
info_box=info_box.astype({"id":int})

In [8]:
info_box.head()

,id,film_name,Directed by,Produced by,Written by,Starring,Music by,Based on,Cinematography,Productioncompany,Release date,Running time,Country,Language,Budget,URL
0,1,Love by the Light of the Moon,Edwin S. Porter,None,None,None,None,None,None,None,\n1901 (1901)\n,None,United States,Silent,None,https://en.wikipedia.org/wiki/Love_by_the_Ligh...
1,2,The Martyred Presidents,Edwin S. Porter,None,None,See below,None,None,Edwin S. Porter,None,1901,1 minutes,United States,English,None,https://en.wikipedia.org/wiki/The_Martyred_Pre...
2,4,Jack and the Beanstalk,George S. FlemingEdwin S. Porter,Edison Studios,None,See below,None,None,Edwin S. Porter,None,1902,10 minutes,United States,English,None,https://en.wikipedia.org/wiki/Jack_and_the_Bea...
3,5,Alice in Wonderland,Cecil HepworthPercy Stow,Cecil M. HepworthHerman Casler (exec. producer...,Lewis Carroll (book)Cecil M. Hepworth,May ClarkCecil M. HepworthMrs. Cecil HepworthN...,None,Alice's Adventures in Wonderland and Through t...,Cecil M. Hepworth,Hepworth Pictures,\n17 October 1903 (1903-10-17)\n,approx. 8:19 minutes,United Kingdom,Silent,None,https://en.wikipedia.org/wiki/Alice_in_Wonderl...
4,6,The Great Train Robbery,Edwin S. Porter,Edwin S. Porter,None,Alfred C. AbadieBroncho Billy AndersonJustus D...,None,"""The Great Train Robbery"" by Scott Marble",Jacob Blair SmithEdwin S. Porter,None,"\nDecember 1, 1903 (1903-12-01)\n",12 minutes (at 18 frame/s),United States,SilentEnglish intertitles,$150[1],https://en.wikipedia.org/wiki/The_Great_Train_...


In [9]:
comd_data=info_box.merge(data,left_on="id",right_index=True)

In [10]:
x=db["ThirdDataSet"].find()
title=pd.DataFrame(x)["title"]
title.values
comd_data.loc[comd_data[comd_data["film_name"].isnull()].index,"film_name"]=title.values

In [11]:
comd_data.head()

,id,film_name,Directed by,Produced by,Written by,Starring,Music by,Based on,Cinematography,Productioncompany,...,Plot_tokenized,Title_lemmatize,Intro_lemmatize,Plot_lemmatize,Title_sw,Intro_sw,Plot_sw,Title_punc,Intro_punc,Plot_punc
0,1,Love by the Light of the Moon,Edwin S. Porter,None,None,None,None,None,None,None,...,"['the', 'moon', 'painted', 'with', 'a', 'smili...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['love', 'by', 'the', 'light', 'of', 'the', 'm...","['the', 'moon', 'painted', 'with', 'a', 'smili...","['love', 'light', 'moon', 'wikipedia']","['love', 'light', 'moon', '1901', 'film', 'edw...","['moon', 'painted', 'smiling', 'face', 'hang',...","['love', 'light', 'moon', 'wikipedia']","['love', 'light', 'moon', '1901', 'film', 'edw...","['moon', 'painted', 'smiling', 'face', 'hang',..."
1,2,The Martyred Presidents,Edwin S. Porter,None,None,See below,None,None,Edwin S. Porter,None,...,"['the', 'film', 'just', 'over', 'a', 'minute',...","['the', 'martyred', 'president', 'wikipedia']","['the', 'martyred', 'president', 'is', 'a', '1...","['the', 'film', 'just', 'over', 'a', 'minute',...","['martyred', 'president', 'wikipedia']","['martyred', 'president', '1901', 'american', ...","['film', 'minute', 'long', 'composed', 'two', ...","['martyred', 'president', 'wikipedia']","['martyred', 'president', '1901', 'american', ...","['film', 'minute', 'long', 'composed', 'two', ..."
2,4,Jack and the Beanstalk,George S. FlemingEdwin S. Porter,Edison Studios,None,See below,None,None,Edwin S. Porter,None,...,"['in', 'this', 'earliest', 'known', 'adaptatio...","['jack', 'and', 'the', 'beanstalk', '1902', 'f...","['jack', 'and', 'the', 'beanstalk', 'is', 'a',...","['in', 'this', 'earliest', 'known', 'adaptatio...","['jack', 'beanstalk', '1902', 'film', 'wikiped...","['jack', 'beanstalk', '1902', 'american', 'sho...","['earliest', 'known', 'adaptation', 'classic',...","['jack', 'beanstalk', '1902', 'film', 'wikiped...","['jack', 'beanstalk', '1902', 'american', 'sho...","['earliest', 'known', 'adaptation', 'classic',..."
3,5,Alice in Wonderland,Cecil HepworthPercy Stow,Cecil M. HepworthHerman Casler (exec. producer...,Lewis Carroll (book)Cecil M. Hepworth,May ClarkCecil M. HepworthMrs. Cecil HepworthN...,None,Alice's Adventures in Wonderland and Through t...,Cecil M. Hepworth,Hepworth Pictures,...,"['alice', 'follows', 'a', 'large', 'white', 'r...","['alice', 'in', 'wonderland', '1903', 'film', ...","['it', 'is', 'now', 'available', 'from', 'seve...","['alice', 'follows', 'a', 'large', 'white', 'r...","['alice', 'wonderland', '1903', 'film', 'wikip...","['available', 'several', 'source', 'included',...","['alice', 'follows', 'large', 'white', 'rabbit...","['alice', 'wonderland', '1903', 'film', 'wikip...","['available', 'several', 'source', 'included',...","['alice', 'follows', 'large', 'white', 'rabbit..."
4,6,The Great Train Robbery,Edwin S. Porter,Edwin S. Porter,None,Alfred C. AbadieBroncho Billy AndersonJustus D...,None,"""The Great Train Robbery"" by Scott Marble",Jacob Blair SmithEdwin S. Porter,None,...,"['the', 'film', 'opens', 'with', 'two', 'bandi...","['the', 'great', 'train', 'robbery', '1903', '...","['in', '1990', 'the', 'great', 'train', 'robbe...","['the', 'film', 'open', 'with', 'two', 'bandit...","['great', 'train', 'robbery', '1903', 'film', ...","['1990', 'great', 'train', 'robbery', 'wa', 's...","['film', 'open', 'two', 'bandit', 'breaking', ...","['great', 'train', 'robbery', '1903', 'film', ...","['1990', 'great', 'train', 'robbery', 'wa', 's...","['film', 'open', 'two', 'bandit', 'breaking', ..."


## TSV FILES

In [138]:
for i in comd_data.index.values:
    comd_data.loc[i:i].to_csv("./tsvfiles/tsv_file{}.txt".format(i),sep="\t")
    

In [12]:
collection=db["Box-infoandRest"]

In [139]:
json_data=comd_data.to_json(orient="records")
collection.insert_many(json.loads(json_data))



In [14]:
search_engine_data=comd_data[["Title_punc","Intro_punc","Plot_punc"]]

In [15]:
for i in search_engine_data.columns :
    print(i)
    for idx in (search_engine_data.index):
        search_engine_data.loc[idx,i]=ast.literal_eval(search_engine_data.loc[idx,i])
        

Title_punc


C:\Users\vedat\Anaconda3\envs\DataScience\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\vedat\Anaconda3\envs\DataScience\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Intro_punc
Plot_punc


In [155]:
raw=search_engine_data.to_json(orient="records")

collection1=db["search_enginedata"]
collection1.insert_many(json.loads(raw))

In [16]:
search_engine_data["new"]= search_engine_data["Intro_punc"] + search_engine_data["Plot_punc"]

C:\Users\vedat\Anaconda3\envs\DataScience\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
inverted_index=defaultdict(list)
for i in (search_engine_data.index):
    for data in set(search_engine_data.loc[i,"new"]):
        inverted_index[data].append("document_{}".format(i))
        

In [71]:
with open("inverted_index.txt","w")as f:
    f.write(json.dumps(inverted_index))
    

In [19]:
#tf-idf calculation
inverted_index2=dict()
for keys in inverted_index:
    idf=1+np.log(search_engine_data.shape[0]/len(inverted_index[keys]))
    inverted_index2[keys]={documents:idf*(search_engine_data.loc[int(documents[9:]),"new"].count(keys)/len(search_engine_data.loc[int(documents[9:]),"new"])) for documents in inverted_index[keys]}

    
    

In [72]:
with open("inverted_index2.txt","w")as f:
    f.write(json.dumps(inverted_index2))
    

In [109]:
y=(search_engine_data.loc[26,"new"].count("love")/len(search_engine_data.loc[26,"new"]))
x=1+np.log(search_engine_data.shape[0]/len(inverted_index["love"]))

In [110]:
x*y

0.012903220495447757

In [101]:
inverted_index2["love"]

{'document_0': 0.03714035088554558,
 'document_20': 0.06543776108405648,
 'document_26': 0.012903220495447757,
 'document_27': 0.05153223685369448,
 'document_28': 0.05725804094854942,
 'document_32': 0.012269580203260591,
 'document_33': 0.049078320813042364,
 'document_42': 0.0404174406695643,
 'document_51': 0.05608950950061984,
 'document_54': 0.049970653918734044,
 'document_63': 0.024539160406521182,
 'document_66': 0.0763440545980659,
 'document_70': 0.02020872033478215,
 'document_71': 0.030708223078551648,
 'document_72': 0.07428070177109115,
 'document_84': 0.06318128656391661,
 'document_89': 0.051856338972271175,
 'document_93': 0.026175104433622597,
 'document_95': 0.03053762183922636,
 'document_99': 0.05285357626019947,
 'document_104': 0.027211742232973984,
 'document_113': 0.025928169486135588,
 'document_114': 0.04164221159894504,
 'document_120': 0.06543776108405648,
 'document_123': 0.034789695766207246,
 'document_125': 0.036645146207071634,
 'document_128': 0.0312

## Cosine Similiraties

In [148]:
## stemming the input query
def stemming_query(inputs):
    words=nltk.word_tokenize(inputs)
    words=[word.lower() for word in words if word.isalnum()]
    lemantizer=nltk.stem.WordNetLemmatizer()
    words =[lemantizer.lemmatize(word) for word in words]
    
    return words

#this function returns the documents that they include all the query and 
# queries idf-tf scores in given sequence

def query_score(queries):
    score_dic=[]
    intersection=set(inverted_index[queries[0]])
    counter=0
    #calculates tf-idf for the que
    try:
        for query in queries:
            if counter == 0:
                idf=1+np.log(search_engine_data.shape[0]/len(inverted_index[query]))
                score_dic.append(idf * (queries.count(query)/len(queries)))
                counter +=1 
            else:
                idf=1+np.log(search_engine_data.shape[0]/len(inverted_index[query]))
                score_dic.append(idf * (queries.count(query)/len(queries)))
                intersection=intersection.intersection(inverted_index[query])
        score_dic=np.array(score_dic)
    except ZeroDivisionError as e1:
        print("Query not found in the inverted_index")
        sys.exit(1)
    return (list(intersection),score_dic)
##this function calculates cosine similarities between query and documents 
## and returns list of (document - cos_sim_score)
def cosine_similarities(documents,query_score):
    ranking_list=[]
    for document in documents:
        doc_score=[]
        for query in queries:
            doc_score.append(inverted_index2[query][document])
        doc_score=np.array(doc_score)
        ranking_list.append((document,np.sum(query_score * doc_score)/(np.sqrt(np.sum(np.square(query_score)))*np.sqrt(np.sum(np.square(doc_score))))))
    
    ranking_list=sorted(ranking_list, key=itemgetter(1),reverse=True)
    return ranking_list
## this function prints the related documents ["film_name","Intro","URL"] from our dataframe
def show_related_data(ranked_list):
    doc_num=[]
    for i in ranked_list:
        doc_num.append(int(i[0][9:]))
    dataframe=comd_data.loc[doc_num,["film_name","Intro","URL"]]
    dataframe["Cosine similiraties"]=[i[1] for i in ranked_list]
    return dataframe

In [156]:
queries=stemming_query(input())
result=query_score(queries)
ranked_list=cosine_similarities(result[0],result[1])
show_related_data(ranked_list)

love


,film_name,Intro,URL,Cosine similiraties
27649,Idhu Nijama,sundaram balachander[6] made a grand reentry i...,https://en.wikipedia.org/wiki/Idhu_Nijama_(194...,1.0
4928,The Damned Don't Cry,the damned don't cry is a 1950 american film n...,https://en.wikipedia.org/wiki/The_Damned_Don%2...,1.0
23044,Bakita Byaktigato (Rest is Personal),bakita byaktigato is a bengali film. the film ...,https://en.wikipedia.org/wiki/Bakita_Byaktigato,1.0
25799,The Shaukeens,the project is a remake of the 1982 film shauk...,https://en.wikipedia.org/wiki/The_Shaukeens,1.0
26439,Nirnayam,the plot is loosely based on the american film...,https://en.wikipedia.org/wiki/Nirnayam_(1995_f...,1.0
...,...,...,...,...
26842,Paappi Appacha,paappi appacha is a 2010 indian malayalam-lang...,https://en.wikipedia.org/wiki/Pappy_Appacha,1.0
1001,Such Men Are Dangerous,such men are dangerous (aka the mask of love) ...,https://en.wikipedia.org/wiki/Such_Men_Are_Dan...,1.0
16626,Catfight,catfight is an american action black comedy fi...,https://en.wikipedia.org/wiki/Catfight_(film),1.0
28091,Kalam Vellum,kalam vellum (transl. time will win) is a 1970...,https://en.wikipedia.org/wiki/Kalam_Vellum,1.0
